In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [39]:
from fastai.imports import *
from fastai.structured import *
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *
from fastai.torch_imports import *
from fastai.io import *

import torch.nn as nn

In [40]:
df_train = pd.read_csv('data/MNIST/train.csv')
df_test = pd.read_csv('data/MNIST/test.csv')
df.shape, df_test.shape

((42000, 785), (28000, 784))

In [41]:
# .values convert to np array
y = df_train['label'].values  
X = df_train.drop(['label'], axis=1).values
X_test = df_test.values

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape, X_test.shape

((33600, 784), (8400, 784), (33600,), (8400,), (28000, 784))

In [45]:
mean = X_train.mean()
std = X_train.std()

X_train = (X_train-mean) / std
X_valid = (X_valid-mean) / std
X_test = (X_test-mean) / std

In [46]:
net = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
    nn.LogSoftmax()
).cuda()

In [47]:
md = ImageClassifierData.from_arrays('tmp/', (X_train, y_train), (X_valid, y_valid), test=X_test)

In [48]:
loss = nn.NLLLoss()
metrics = [accuracy]
opt = optim.SGD(net.parameters(), 1e-1, momentum=0.9, weight_decay=1e-3)

In [49]:
set_lrs(opt, 1e-2)

In [50]:
fit(net, md, n_epochs=5, crit=loss, opt=opt, metrics=metrics)

epoch      trn_loss   val_loss   accuracy                     
    0      0.201571   0.204008   0.939405  
    1      0.147354   0.149307   0.953929                     
    2      0.09882    0.124809   0.959286                      
    3      0.093807   0.117243   0.963333                      
    4      0.077324   0.116341   0.962976                      



[array([0.11634]), 0.9629761904761904]

In [51]:
preds = predict(net, md.val_dl)
preds = preds.argmax(1)
np.mean(preds == y_valid)

(8400, 10)

In [54]:
y_preds = predict(net, md.test_dl)
y_preds = y_preds.argmax(1)

In [55]:
import os

os.makedirs('results', exist_ok=True)

my_submission = pd.DataFrame({'ImageId': list(range(1,len(df_test)+1)),
                              "Label": y_preds})
my_submission.to_csv('output/digit-recognizer-fastai-NN-basic.csv', index=False, header=True)